In [51]:
import keras
from keras.models import Model, Sequential
import numpy as np
from keras.optimizers import SGD, Adam
from keras.layers import Dense, LSTM, Input, Reshape, merge

# 1. Loading input data and testing data

In [45]:
X = np.array([[0.2 , 0.3, 0.4, 0.5], [0.3, 0.4, 0.5, 0.6], [0.2, 0.2, 0.3, 0.3]])
Y = np.array([[0.3, 0.4, 0.5, 0.6], [0.4, 0.5, 0.6, 0.7], [0.3, 0.3, 0.3, 0.3]])
noise = np.random.random((3, 4)) 

(3, 4)

In [28]:
new_data = np.array([0.5, 0.2, 0.3, 0.5])
new_data = np.reshape(new_data, (1,4))

# 2. Defining the generator and the discriminator

In [54]:
# try out different numbers of output dimensions
def get_generative_model(hidden_neurons = 12, input_dim = 4):    
    model = Sequential()
    model.add(Dense(hidden_neurons, activation='relu', input_dim = 4))
    predicted = model.add(Dense(4, activation='sigmoid'))
    return model 

In [65]:
first_input = Input(shape=(4, ))
first_dense = Dense(4, )(first_input)

second_input = Input(shape=(4, ))
second_dense = Dense(4,)(second_input)

merge_one = merge([first_dense, second_dense])
model = Model(inputs=[first_input, second_input], outputs = merge_one)

TypeError: __init__() got an unexpected keyword argument 'outputs'

In [57]:
'''
first_input = Input(shape=(4,))
first_dense = Dense(1, )(first_input)

second_input = Input(shape=(2, ))
second_dense = Dense(1, )(second_input)

merge_one = merge([first_dense, second_dense])

third_input = Input(shape=(1, ))
merge_two = merge([merge_one, third_input])

model = Model(inputs=[first_input, second_input, third_input], outputs=merge_two)
model.compile(optimizer=ada_grad, loss='binary_crossentropy',
               metrics=['accuracy'])
'''

"\nfirst_input = Input(shape=(4,))\nfirst_dense = Dense(1, )(first_input)\n\nsecond_input = Input(shape=(2, ))\nsecond_dense = Dense(1, )(second_input)\n\nmerge_one = merge([first_dense, second_dense])\n\nthird_input = Input(shape=(1, ))\nmerge_two = merge([merge_one, third_input])\n\nmodel = Model(inputs=[first_input, second_input, third_input], outputs=merge_two)\nmodel.compile(optimizer=ada_grad, loss='binary_crossentropy',\n               metrics=['accuracy'])\n"

In [31]:
def get_discriminative_model():
    model = Sequential()
    model.add(Dense(12, input_dim=4, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model

# 3. Running through model to get predictions from the generator

In [58]:
generator = get_generative_model()
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator.fit(X, Y)
generated_values = generator.predict(X)

TypeError: Merge can only be called on a list of tensors, not a single tensor. Received: (<tf.Tensor 'generator_input:0' shape=(4,) dtype=float32>, None)

In [33]:
generated_values

array([[ 0.5036608 ,  0.45059842,  0.53698927,  0.47853285],
       [ 0.50947827,  0.43779942,  0.54178965,  0.46556687],
       [ 0.50808102,  0.4705317 ,  0.51671654,  0.47299281]], dtype=float32)

# 4. Creating input for the discriminator

In [38]:
# X = np.reshape(X, (3,4))
# Y = np.reshape(Y, (3,4))
# generated_input_to_discriminator = np.concatenate([X, generated_values], axis = 1)
# generated_input_to_discriminator = np.reshape(generated_input_to_discriminator,(3,8))
generated_input_to_discriminator = np.concatenate([X, generated_values], axis = 1) 
true_input_to_discriminator = np.concatenate([X, Y], axis = 1)

#true_input_to_discriminator = np.reshape(true_input_to_discriminator,(3,8))
#true_input_to_discriminator = Y

Xdisc = np.concatenate((generated_input_to_discriminator, true_input_to_discriminator))
Ydisc = [1] * len(generated_input_to_discriminator) + [0] * len(true_input_to_discriminator)
new_data_disc = np.array([0.2,0.5,0.4, 0.2, 0.3, 0.6, 0.5, 0.3])
#new_data_disc = np.reshape (new_data_disc, (2,8))

In [42]:
Xdisc = Xdisc.tolist()

# 5. Running through model to get output from the discriminator

In [23]:
discriminator = get_discriminative_model()
discriminator.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
discriminator.fit (Xdisc, Ydisc)
discriminator.predict(Xdisc)

Epoch 1/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 2/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 3/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 4/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 5/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 6/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 7/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 8/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 9/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000
Epoch 10/10
6/6 [==============================] - 0s - loss: 0.6933 - acc: 0.5000


array([[ 0.49895456],
       [ 0.49895945],
       [ 0.49895468],
       [ 0.49912736],
       [ 0.49891233],
       [ 0.49938491]], dtype=float32)

# 6. Connecting the discriminator to the generator

In [24]:
def combined_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    # Here generator's output are arrays that are 8 numbers long
    # Discriminator takes inpt that is 16 numbers long
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [25]:
generator_containing_disciminator = combined_model(generator, discriminator)

# ALTERNATIVE 
DISCRIMINATOR THAT TAKES THE GENERATORS OUTPUT IMMEDIATELY

In [281]:
def get_generative_model(hidden_neurons = 12, input_dim=4):    
    model = Sequential()
    model.add(Dense(hidden_neurons, activation='relu', input_dim = 4))
    model.add(Dense(4, activation='sigmoid'))
    #model.add(Reshape((4,1), input_shape=(1,4)))
    return model 

In [272]:
def get_discriminative_model():
    model = Sequential()
    model.add(Dense(12, input_dim=4, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model

In [309]:
new_data = np.reshape(new_data, (1,4))

In [311]:
generator = get_generative_model()
generated_values = generator.predict(new_data)
new_data.shape

(1, 4)

In [285]:
real_input = np.reshape(Y, (1,4))
Xdisc = np.concatenate((generated_values, real_input))
Ydisc = [1] * len(generated_values) + [0] * len(real_input)
new_data = np.array([0.6, 0.6, 0.6, 0.6])
new_data = np.reshape(new_data, (1,4))

In [287]:
discriminator = get_discriminative_model()
discriminator.compile(loss='binary_crossentropy', optimizer='sgd')
discriminator.fit (Xdisc, Ydisc)
discriminator.predict(new_data)

Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.6932


array([[ 0.49939361]], dtype=float32)

## Combining the discriminator with the generator

In [291]:
def combined_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [292]:
generator_containing_disciminator = combined_model(generator, discriminator)

In [356]:
Ydisc1 = [1]

In [358]:
generator_containing_disciminator.fit(X, Ydisc1)

AttributeError: 'int' object has no attribute 'shape'

In [295]:
# # 8. Setting optimizers for the generator and the discriminator
d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
generator.compile(loss='binary_crossentropy', optimizer='sgd')
generator_containing_disciminator.compile(loss='binary_crossentropy', optimizer=g_optim)
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)

# Getting losses

In [353]:
def get_losses(i):
    epoch = 0
    print ('Epoch:', epoch+1)
    d_losses = []
    g_losses = []
    for batchno in range(3):
        inputdata = X
        generated_audio = generator.predict(inputdata)
        real_audio = Y
        Xdisc = np.concatenate((real_audio, generated_audio))
        Ydisc = [1] * len(real_audio) + [0] * len(generated_audio)
        d_loss = discriminator.fit(Xdisc, Ydisc)  # later change to train on batch
        d_losses.append(d_loss)
        discriminator.trainable = False
        g_loss = generator_containing_disciminator.fit(inputdata, len(real_audio))
        #print g_loss
    epoch =+ 1
    return True

In [352]:
get_losses(0)

('Epoch:', 1)
Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 6/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 7/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 8/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 9/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 10/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 1/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 2/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 3/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 4/10
2/2 [==============================] - 0s - loss: 0.6932
Epoch 5/10
2/2 [=================

True

In [ ]:

        g_loss = generator_containing_disciminator.train_on_batch(inputdata, [1]*batch_size)
        # take into account example data also. input audio - example of laugter - paper about images from Daniel. 
        
        # 9.8 Calculate losses for the generator
        g_losses.append(g_loss)
        
        # I thought the discriminator was set to not trainable already in step 9.7??
        discriminator.trainable = True
        # Print how many batches have been trained on 
        sys.stdout.write(' + batch: ' + str(index+1) + '/' + str(n_batches) + '\r')
        # Everything from the buffer will be written in the terminal 
        sys.stdout.flush()
    return d_losses, g_losses
